In [7]:
import numpy as np
from scipy.stats import norm

# set precision
%precision 3

'%.3f'

In [2]:
# Given data
data = np.array([
    [184, 0, 0, 1, 2.1, 0],
    [243, 0, 0, 1, 0, 1],
    [222, 0, 1, 1, 1.2, 1],
    [309, 0, 0, 1, 0, 0],
    [198, 0, 1, 1, 1.6, 0],
    [308, 0, 0, 0, 0, 1],
    [234, 0, 0, 0, 0, 1],
    [197, 0, 0, 0, 0, 0],
    [266, 0, 0, 1, 2.2, 0],
    [195, 0, 1, 0, 0, 1],
    [223, 0, 1, 0, 0, 1],
    [309, 0, 1, 0, 0, 1],
    [233, 1, 0, 0, 2.3, 1],
    [289, 1, 0, 0, 1, 0],
    [235, 0, 1, 0, 0, 1]
])


In [48]:
# Extract Class column
class_column = data[:, -1]

# Calculate P(Class = 0)
p_class_0 = np.sum(class_column == 0) / len(class_column)

print("P(Class = 0):", p_class_0)

P(Class = 0): 0.4


In [9]:
np.sum(class_column==0)

6

In [38]:
# Extract relevant columns (FBS and Class)
fbs_column = data[:, 1]
rest_ecg_col = data[:,2]
exang_col = data[:,3]
class_column = data[:, -1]

# Calculate P(FBS=1 | Class=1)
p_fbs_1_given_class_1 = np.sum((fbs_column == 1) & (class_column == 1)) / np.sum(class_column == 1)

p_r1_given_0 = np.sum((rest_ecg_col == 1) & (class_column == 0)) / np.sum(class_column == 0)
 

p_e0_g_0 = np.sum((exang_col==0) & (class_column == 0)) / np.sum(class_column == 0)
p_r1_given_0, p_e0_g_0

(0.167, 0.333)

In [34]:
p_fbs_0_given_class_0 = np.sum((fbs_column == 1) & (class_column == 0)) / np.sum(class_column == 0)
print(f"P(FBS=1 | Class=0): {p_fbs_0_given_class_0:.3f}", )


P(FBS=1 | Class=0): 0.167


In [31]:
# Extract relevant columns (excluding Class)
X_train = data[:, :-1]
y_train = data[:, -1]

# Unknown vector
x_unknown = np.array([193, 1, 1, 0, 3.4])

c_x_unknown = np.array([193, 3.4])


In [32]:
# Separate data by class
class_data = {}
for class_label in np.unique(y_train):
    class_data[class_label] = X_train[y_train == class_label]
class_data

{0.000: array([[184. ,   0. ,   0. ,   1. ,   2.1],
        [309. ,   0. ,   0. ,   1. ,   0. ],
        [198. ,   0. ,   1. ,   1. ,   1.6],
        [197. ,   0. ,   0. ,   0. ,   0. ],
        [266. ,   0. ,   0. ,   1. ,   2.2],
        [289. ,   1. ,   0. ,   0. ,   1. ]]),
 1.000: array([[243. ,   0. ,   0. ,   1. ,   0. ],
        [222. ,   0. ,   1. ,   1. ,   1.2],
        [308. ,   0. ,   0. ,   0. ,   0. ],
        [234. ,   0. ,   0. ,   0. ,   0. ],
        [195. ,   0. ,   1. ,   0. ,   0. ],
        [223. ,   0. ,   1. ,   0. ,   0. ],
        [309. ,   0. ,   1. ,   0. ,   0. ],
        [233. ,   1. ,   0. ,   0. ,   2.3],
        [235. ,   0. ,   1. ,   0. ,   0. ]])}

In [23]:
# Calculate mean and standard deviation for Chol and Oldpeak in each class
class_params = {}
for class_label, class_samples in class_data.items():
    class_params[class_label] = {
        'mean': np.mean(class_samples, axis=0),
        'std': np.std(class_samples, axis=0, ddof=1),
    }
class_params

{0.000: {'mean': array([2.405e+02, 1.667e-01, 1.667e-01, 6.667e-01, 1.150e+00]),
  'std': array([54.01 ,  0.408,  0.408,  0.516,  0.987])},
 1.000: {'mean': array([2.447e+02, 1.111e-01, 5.556e-01, 2.222e-01, 3.889e-01]),
  'std': array([38.643,  0.333,  0.527,  0.441,  0.819])}}

In [47]:
# Calculate likelihood for continuous features (Chol and Oldpeak) for class 0
likelihood_chol = norm.pdf(x_unknown[0], loc=class_params[0]['mean'][0], scale=class_params[0]['std'][0])
likelihood_oldpeak = norm.pdf(x_unknown[4], loc=class_params[0]['mean'][4], scale=class_params[0]['std'][4])

likelihood_chol, likelihood_oldpeak

(0.005, 0.030)

In [41]:
0.167 * 0.005 * 0.913 * 0.167 * 0.333

0.000

In [53]:
#p_e0_g_0 = np.sum((exang_col==0) & (class_column == 0)) / np.sum(class_column == 0)

def calc_col(col_data, col_comp, class_comp):
    return np.sum((col_data==col_comp) & (class_column == class_comp)) / np.sum(class_column == class_comp)

In [52]:
lchol_class1 = norm.pdf(x_unknown[0], loc=class_params[1]['mean'][0], scale=class_params[1]['std'][0])
lpeak_class1 = norm.pdf(x_unknown[4], loc=class_params[1]['mean'][4], scale=class_params[1]['std'][4])

lchol_class1, lpeak_class1

(0.004, 0.001)

In [57]:
calc_col(fbs_column, 1, 1)

0.111